# Explicação 

Referência :  https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

O processo de aprendizado por reforço envolve:

1. Observar o ambiente
2. Detectar como agir usando alguma estrategia
3. Agir de acordo com essa estrategia
4. Avaliar o resultado da ação e penalizar ou recompensar 
5. Aprender atraves das experiencias refinando nossa estrategia 
6. Iterar ate uma estrategia otima ser encontrada

O problema apresentado é uma simplicação de um táxi autônomo,
o objetivo do táxi é pegar um passageiro e o transportar para um determinado
local no menor tempo possivel, cuidando para não inflingir nenhuma regra de trafico. 

Dentro do contexto de RL (Reinforcement Learning) existe 3 conceitos principais:
    Ações: É o conjunto de ações que se pode realizar em determinada momento (estado).
    Para o caso do problema do taxi, existe 6 ações possiveis:
        1. ir para a dereita
        2. ir para a esquerda
        3. ir para tras
        4. ir para frente 
        5. Pagar o passageiro
        6. Deixar o passageiro

    Estado: No contexto de aprendizado por reforço o agente se encontra em um estado
    e deve realizar uma ação para ir para outro estado.

    Recompensas: É o valor dado ao agente pela sua ação em determinado estado, esse 
    valor é positivo caso a ação seja uma recompensa e negativo em caso de penalização.

O objetivo do aprendizado por reforço é encontrar uma sequencia de ações que maximize
a recompensa do agente.

No problema do táxi a seguir,onde existe 4 locais
e o objetivo do agente é pegar o passageiro em um local (representado pela cor azul) e transportar
para outro (representado pela cor rosa). O agente recebe +20 pontos caso ele consiga deixar o passageiro
no local exato e perde 1 ponto cada vez que ele realiza uma ação, há tambem uma penalidade de 10 pontos para o caso 
do agente pegar ou deixar o passageiro em um local errado.

# Código

In [2]:
# O quadrado fica amarelo se o táxi esta sem passageiro e verde com passageiro.

import gym

env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Espaço de ações {}".format(env.action_space))
print("Espaço de estados {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Espaço de ações Discrete(6)
Espaço de estados Discrete(500)


In [5]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("Estado:", state)

env.s = state
env.render()

Estado: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
# {ação: [(probabilidade, proximo estado, recompensa, finalizado)]}.

env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [8]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 627
Penalties incurred: 211


In [11]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.5)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 627
State: 0
Action: 5
Reward: 20


# Introdução ao Q-learning



In [14]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [15]:
import random
import numpy as np
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [25]:
state = env.reset()
epochs, penalties, reward = 0, 0, 0
frames = []
done = False
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 7
State: 410
Action: 5
Reward: 20
